# PyMOL Remote demo

### 1. [Prerequisite] Server side (where PyMOL is running)

This demo should be run on the `client` side, i.e. the machine where you want to run your Python code.
To run it, you will need to have run the following command on the `server` side, i.e. the machine where PyMOL is running:

1. **Server side (where PyMOL is running)**
```bash
# Start PyMOL with the RPC server as explained in 2.1.
# By default, it will listen on localhost:9123
pymol_remote
```

2. **Client side (on your remote machine)**
Connect to the machine (e.g. a cloud server, your HPC, etc.) where you want to run your Python code via SSH and set up port forwarding.
You will need to use the same port number as the one you used when starting PyMOL on the server side, by default 9123.
```bash
# Set up SSH port forwarding (run this in a terminal)
ssh -R 9123:localhost:9123 username@server_address
```

For more details on how to use `pymol_remote`, see the official usage instructions on the GitHub page or [Martin Buttenschön's short blog post](https://www.blopig.com/blog/2024/11/controlling-pymol-from-afar/) on controlling PyMOL from afar.

### 2. Client side (where you want to run your Python code)

In [ ]:
from pymol_remote.client import PymolSession

# NOTE: When you run `pymol_remote` on the server side, it will print a likely guess of your
#  IP address (and the correct port number, 9123 by default).
HOSTNAME = "localhost"
PORT = 9123
pymol = PymolSession(hostname=HOSTNAME, port=PORT)

# You can now send commands to PyMOL
pymol.fetch("6lyz")
pymol.do("remove solvent")
pymol.do("set valence, on")
pymol.get_state(format="cif")

In [3]:
# Make pymol look pretty (Courtesy of Kate Fie: https://www.blopig.com/blog/2024/12/making-pretty-pictures-in-pymol-v2/)
from pymol_remote.style import make_pymol_pretty

make_pymol_pretty(pymol)

In [ ]:
pymol.print_help()

Get help on a specific command

In [ ]:
pymol.help("fetch")

In [ ]:
pymol.help("get_state")

Show all the available commands

In [ ]:
pymol.help()